# Genetic Algorithms

We will use a the context of the knapsack problem to discuss the steps of a genetic algorithm. The knapsack problem is where we have p different items each with a different weight and value. Our strategy will be to choose which items to take and which to leave behind. We will define our strategy as a list of binary values. 1 represents we take item in our knapsack, 0 represents the decision to leave it behind.

In the case where p = 10 a potential solution is as follows
[0,1,0,0,0,1,1,1,0]

This means we take item 2,7,8,and 9 and leave the rest behind.

We setup the knapsack problem below:

In [2]:
#potential solution [0,1,0,0,0,1,1,1,0]

import random

# knapsack parameters
NUM_ITEMS = 10
MAX_WEIGHT = 2
values = [random.uniform(1,20) for _ in range(NUM_ITEMS)]
weights = [random.random() for _ in range(NUM_ITEMS)]
print(f"item#    weight  value")
for i,w,v in zip(range(NUM_ITEMS),weights,values):
  print(f"{i+1} \t {w:0.2f} \t {v:0.2f}")

item#    weight  value
1 	 0.45 	 9.76
2 	 0.75 	 3.39
3 	 0.38 	 17.50
4 	 0.27 	 2.21
5 	 0.66 	 15.10
6 	 0.26 	 14.36
7 	 0.31 	 6.42
8 	 0.68 	 5.96
9 	 0.76 	 3.10
10 	 0.51 	 4.46


# Genetic Algorithm Steps

The flow of a genetic algorithm goes as follows:

1. [Initialize Population](#init_population) Create the initial population of N randomly generated solutions<br>
2. [Reproduction](#reproduction) Create m new solutions<br>
    2.1 [Selection](#selection) Select m parents to reproduce form the current N solutions<br>
    2.2 [Crossover](#crossover) Generate m children from the m parents<br>
    2.3 [Mutation](#mutation) Randomly alter parts of the new children<br>
3. [Survival](#survival) Choose N of the N+m solutions for the new set of possible solutions (survival of the fittest)<br>
4. Repeat from 2.

<a id='init_population'></a>
# Create the initial population


Our first step is to create the population. In our case we will represent the population as a list of N randomly generated solutions. So, before we can worry about generating an entire population we should focus on generating a single, randomly generated solution. 

In [3]:
def random_knapsack_solution(num_items):
  return [random.randint(0,1) for _ in range(num_items)]
random_knapsack_solution(NUM_ITEMS)

[1, 1, 1, 0, 1, 1, 0, 0, 1, 0]

Now that we can generate random solutions to the knapsack problem, we can generate an entire population (a fixed set) of solutions.

In [4]:
POPULATION_SIZE = 50
def generate_knapsack_population(num_items, population_size):
    population = [random_knapsack_solution(num_items) for _ in range(population_size)]
    return population
generate_knapsack_population(NUM_ITEMS,POPULATION_SIZE)

[[0, 1, 1, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
 [0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
 [1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
 [1, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 1, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 1, 0, 1, 0, 0],
 [1, 1, 1, 0, 0, 0, 1, 1, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
 [0, 1, 1, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 0, 1, 1, 0, 0, 1],
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 1, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 1, 0, 1, 0, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 1,

# Fitness

We need to be able to rank our solutions, so we can determine which one is the best. We will use our fitness funciton to do this. Our fitness will be the cross product our decision for each item $\delta_i$, the weight of each item $w_i$, and the value of each item $v_i$.
Our total weight will be

$W = \displaystyle\sum_{i=1}^{p}\delta_i \cdot w_i$

Our total value will be

$V = \displaystyle\sum_{i=1}^{p}\delta_i \cdot v_i$



We also need to decide what to do if we go over weight. In our case if we go over weight, we will return the negative of how far overweight we are.


In [ ]:
def knapsack_fitness(solution, values, weights, max_weight):
  total_weight = 0
  total_value = 0
  for choice,weight,value in zip(solution,weights,values):
    total_weight += choice*weight
    total_value += choice*value
  return total_value if total_weight <= max_weight else max_weight-total_weight

test_solution = [1,0,1,0]
test_values = [10,15,20,3]
test_weights = [1,2,3,4]
max_weight = 4
knapsack_fitness(test_solution,test_values,test_weights,max_weight)

30

[[1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 0, 1, 1, 0, 1, 1],
 [0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 0, 1, 0, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
 [0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
 [1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 1, 1, 0, 1, 1, 0],
 [1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 1, 0, 0, 1, 0],
 [1, 1, 1,

<a id='reproduction'></a>
# Reproduction
Reproduction is the act of generating new solutions (adding children) using parts of the currentsolutions in the population.  This is vital to the genetic algorithm finding the optimal fitness value.Without reproduction, the GA would eventually converge to one of the solutions with the highestfitness  level  in  the  initial  population.   Since  these  were  randomly  generated  there  is  a  very  lowchance that this would be an optimal solution to the problem.  Reproduction consists of two parts:the generation of the new solution using current solutions (**crossover**) and the chance that the newsolution is mutated during creation (**mutation**).

<a id='selection'></a>
## Selection
The purpose of reproduction is to naturally create new solutions using the current solutions in thepopulation.  This is supposed to mimic parents passing their genetic material to their children.  The way children are created is by first selecting a set of parents (**Selection**) and combining parts of eachparent to create a new child (or children) solutions (Crossover).  Selection of parents can be done many ways but the bottom line is that parents are randomly selected from the current population.

Once the parents are selected the children are created using Crossover.  Crossover can also be donemany  ways  but  the  bottom  line  here  is  that  some  portion  of  each  parent  is  used  to  create  thechildren.  Through multiple recombination the new solutions could take on the best parts of some of the current solutions to create better solutions that did not initially exist.
Selection Methods:
### Roulette  Selection:
Gives  each  solution  in  the  current  population  the  same  probability  ofbeing selected as a parent.



In [ ]:
NUM_ITEMS = 10
MAX_WEIGHT = 2
POPULATION_SIZE = 50

population = generate_knapsack_population(NUM_ITEMS,POPULATION_SIZE)

def roulette_selection(population):
  return random.sample(population,2)

parent1,parent2 = roulette_selection(population)
print(f"Parent 1 is {parent1}")
print(f"Parent 2 is {parent2}")


Parent 1 is [0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
Parent 2 is [0, 0, 1, 1, 0, 0, 1, 0, 0, 0]


### Weighted Roulette Selection: Weighted roulette selection will give solutions with higher fitnessa higher chance of being selected proportional to their fitness.  
* Note:  be careful of negative fitness values.
In this case we transformed the data and shifted everything right by the magnitude of the min value.


In [ ]:
#test of the weights
pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]
print(pop_fitness[:10])
selection_weights = [fitness - min(pop_fitness) for fitness in pop_fitness]
print("weights")
selection_weights[:10]

[-1.6663739237144823, 33.94454600931266, -0.7112787482059799, 33.246024596687576, 16.8626268738425, -0.4793473255163687, -0.5614006162567104, -0.7215335035854027, -0.541482369919585, -0.8974146452014025]
weights


[0.7845522176084168,
 36.395472150635555,
 1.7396473931169192,
 35.69695073801047,
 19.3135530151654,
 1.9715788158065304,
 1.8895255250661886,
 1.7293926377374964,
 1.9094437714033141,
 1.5535114961214966]

In [ ]:
def weighted_roulette_selection(population):
  pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]
  prob_weights = [fitness - min(pop_fitness) for fitness in pop_fitness]
  return random.choices(population,prob_weights,k=2)
parent1,parent2 = weighted_roulette_selection(population)
print(f"Parent 1 is {parent1}")
print(f"Parent 2 is {parent2}")

Parent 1 is [0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
Parent 2 is [0, 1, 1, 0, 0, 1, 1, 0, 0, 0]




*   Ranked Roulette Selection:  Rank solutions by fitness and then weight probability based on the rank. This reduces the effect of the magnitude of the difference in fitness


In [ ]:
def rank_fitness(pop_fitness):
  return [sorted(pop_fitness).index(fit) for fit in pop_fitness]

def ranked_roulette_selection(population):
  pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]
  pop_ranks = rank_fitness(pop_fitness)
  return random.choices(population,pop_ranks,k=2)

parent1,parent2 = weighted_roulette_selection(population)
print(f"Parent 1 is {parent1}")
print(f"Parent 2 is {parent2}")

Parent 1 is [0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
Parent 2 is [0, 0, 1, 1, 0, 0, 1, 0, 0, 0]


We're going to do something a bit cheesy and do a pythonic switch statement. It's realy convenient if you want be able to interchangeably switch between different functions. The important thing to note is that the key must be unique and non-mutable. I used strings here for readability, but convention is to use integers. This only works if all of the functions have the same input parameters. You can get around different parameter dimenisons by adding default None parameters at the cost of readability.

In [ ]:
def parent_selection(population,type="roulette"):
  selection_switch = {
      "roulette" : roulette_selection,
      "weighted" : weighted_roulette_selection,
      "ranked" : ranked_roulette_selection
      }
  return selection_switch[type](population)

selection_type = "ranked"
parent1,parent2 = parent_selection(population,selection_type)
print(f"Parent 1 is {parent1}")
print(f"Parent 2 is {parent2}")

Parent 1 is [1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
Parent 2 is [1, 1, 0, 0, 1, 0, 0, 1, 0, 0]


<a id='crossover'></a>
## Crossover

The type of crossover depends on the type of strategies that you use to represent the parents. If you are using a list the following are great options

### 1 Point Crossover:  
1. Randomly draw an index between 0 and p-1 where p is the length of the solution array 
2. Take all parts of parent 1 before that point and all parts of parent 2 after that point to create child 1.  
3. Create child 2 starting with parent 2 and ending with parent 1.

#### Example
Parent 1: [0, 1, 1, 1, 0, 0, 0, 1, 0, 1]

Parent 2: [1, 1, 0, 1, 0, 0, 0, 0, 1, 1]

>Randomly draw an index between 0 and p-1 where p is the length of the solution array 

random index = 4

>Take all parts of parent 1 before that point


Child1 inherits from Parent 1: [0, 1, 1, 1, 0,\_,\_,\_,\_,\_]


>and all parts of parent 2 after that point

Child1 inherits from Parent 2: [\_,\_,\_,\_,\_,0, 0, 0, 1, 1]


> to create child 1.

Child 1: [0, 1, 1, 1, 0,0, 0, 0, 1, 1]

> Create child 2 starting with parent 2 and ending with parent 1.

Child2 inherits from Parent 1: [\_,\_,\_,\_,\_, 0, 0, 1, 0, 1]

Child2 inherits from Parent 2: [1, 1, 0, 1, 0,\_,\_,\_,\_,\_]

Child 2: [0, 1, 1, 1, 0,0, 0, 0, 1, 1]


In [ ]:
def one_point_crossover(parent1,parent2):
  i = random.randrange(len(parent1))
  child1 = parent1[:i]+parent2[i:]
  child2 = parent1[i:]+parent2[:i]
  return [child1,child2]
parent1 = [0, 1, 1, 1, 0, 0, 0, 1, 0, 1]
parent2 = [1, 1, 0, 1, 0, 0, 0, 0, 1, 1]
child1,child2 = one_point_crossover(parent1,parent2)
print(f"Child 1 is {child1}")
print(f"Child 2 is {child2}")

Child 1 is [0, 1, 1, 1, 0, 0, 0, 1, 0, 1]
Child 2 is [1, 1, 1, 0, 1, 0, 0, 0, 0, 1]


* k Point Crossover:  Same as 1 point but you draw multiple points randomly and alternate theparent used between the points.


In [ ]:
def k_point_crossover(parent1,parent2,k):
  l =len(parent1)
  if k > l:
    print("Bad input")
    return -2
  indices = sorted(random.choices(range(l+1),k=k))
  print(indices)
  #flip it back and forth to decide which parent
  switch = 1
  child1 = []
  child2 = []
  for i,j in zip([0]+indices,indices+[l]):
    if switch:
      child1 += parent1[i:j]
      child2 += parent2[i:j]
    else:
      child1 += parent2[i:j]
      child2 += parent1[i:j]
    #xor with 1 to flip back and forth
    switch ^=1
  return child1,child2

print(f"Parent 1 is {parent1}")
print(f"Parent 2 is {parent2}")
child1,child2 = k_point_crossover(parent1,parent2,4)
print(f"Child 1 is {child1}")
print(f"Child 2 is {child2}")

Parent 1 is [0, 1, 1, 1, 0, 0, 0, 1, 0, 1]
Parent 2 is [1, 1, 0, 1, 0, 0, 0, 0, 1, 1]
[0, 1, 4, 4]
Child 1 is [1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
Child 2 is [0, 1, 0, 1, 0, 0, 0, 0, 1, 1]


Debugging tip. We can confirm that this works since the sum of the cells for the parents should be the same as the sum of the cells for the children.

* Arithmetic Crossover:  This is used for real valued solutions (not binary).  Draw a point in the solution at random then use parent 1 before that point and the average between parent 1and 2 after that point.  Child 2 instead uses parent 2 before averaging.

Try writing a function fo this one on your own for practice.

<a id='mutation'></a>
# Mutation

When reproduction occurs there is a small chance that the genetic material is not copied correctlyand a mutation occurs.  With some small probability, there is a chance that each value in a solution can change slightly.  In a GA this is essential to avoiding local maxima.  The goal is to produce newsolutions that are similar to the current solutions but to allow enough change to explore other areasof the solution space.  Think back to neighbor generation in Simulated Annealing.

Our mutation scheme: For each of our choices we will draw a random number and change the bit at that location if that number is less than or equal to our probability.

For integral strategies we can add a gaussian variable and that will serve us well.

In [ ]:
def mutate_knapsack(solution,prob):
  return [s^1 if random.random() <prob else s for s in solution]
print(parent1)
mutate_knapsack(parent1,0.8)


[0, 1, 1, 1, 0, 0, 0, 1, 0, 1]


[1, 0, 0, 0, 1, 1, 1, 0, 1, 0]

# Generate Children
We now just have to put together our functions for selection, crossover, and mutation to generate our children

In [ ]:
def generate_children(population,num_children,mutate_prob):
  children = []
  for _ in range(num_children//2):
    parent1,parent2 = parent_selection(population)
    children += one_point_crossover(parent1,parent2)
  return [mutate_knapsack(child,mutate_prob) for child in children]

<a id='survival'></a>
# Survival
After  new  solutions  are  generated  the  population  needs  to  be  pruned  back  down  to  the  originalsize.  Populations cannot sustain overpopulation so survival of the fitest occurs naturally.  This isemulated in GA through different survival methods.  Ultimately, the GA takes the original N andthe m children and choose only N to keep.  There are a few different ways to do this:
## Tournament Selection:
 * Pick 2 of the N + m strategies (solutions) at random (with replacement)
 * Put the bettwer solution of the two into the new population
 * Repeat N times
 > * results in better solutions
 > * Even if you choose 2 bad solutions, you get rid of the worse one

 ## Elitism
 * Place the top X\% of the total population in the new population.
 * Replace the others using different survival method

 You can engineer your own survival methods by tweaking with the algorihtm and the rules. These can be changed out pretty easily.

In [ ]:
NUM_ITEMS = 10
MAX_WEIGHT = 2
POPULATION_SIZE = 50
population = generate_knapsack_population(NUM_ITEMS,POPULATION_SIZE)
NUM_CHILDREN = 50
MUTATE_PROB = 0.2
children = generate_children(population,NUM_CHILDREN,MUTATE_PROB)
new_population = population + children

pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]

def tournament_selection(new_population,population_size,pop_fitness):
  trimmed_population = []
  for _ in range(population_size):
    (strat1,fit1),(strat2,fit2) = random.sample(list(zip(new_population,pop_fitness)),2)
    trimmed_population.append(strat1) if fit1>fit2 else trimmed_population.append(strat2)
  return trimmed_population
tournament_selection(new_population,POPULATION_SIZE,pop_fitness)[:10]

[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 1],
 [1, 0, 1, 1, 0, 0, 0, 1, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 1, 1, 0],
 [1, 0, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0]]

#Putting it all together
Let's just copy and paste the steps down here for the sake of readability. I know I've been copying and pasting things all over. That's not the best practices for efficient code, but it's perfectly acceptable for exploration and education.

# Genetic Algorithm Steps
1. Create Initial Population: N randomly generated solutions
2. Reproduction
* Selection: Select m parents to reproduce form the current N solutions
* Crossover: Generate m children from the m parents
* Mutation: Randomly alter parts of the new children
3. Survival: choose N of the N+m solutions for the new set of possible solutions (survival of the fittest)
4. Repeat from step 2

In [ ]:
import random

# Create knapsack parameters
NUM_ITEMS = 10
MAX_WEIGHT = 2
values = [random.uniform(1,20) for _ in range(NUM_ITEMS)]
weights = [random.random() for _ in range(NUM_ITEMS)]
print(f"item#    weight  value")
for i,w,v in zip(range(NUM_ITEMS),weights,values):
  print(f"{i+1} \t {w:0.2f} \t {v:0.2f}")
#set up the GA
POPULATION_SIZE = 50
#initial population
initial_population = generate_knapsack_population(NUM_ITEMS,POPULATION_SIZE)
#rules for reproduction
print(f"starting maximum: {max([knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in initial_population])}") 
NUM_CHILDREN = 50
MUTATE_PROB = 0.2

def ga_iteration(population, num_children,mutate_prob):
  children = generate_children(population,NUM_CHILDREN,MUTATE_PROB)
  new_population = population + children
  pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]
  return tournament_selection(new_population,POPULATION_SIZE,pop_fitness)
current_population = initial_population

NUM_ITER = 10000
for _ in range(NUM_ITER):
  current_population = ga_iteration(current_population,NUM_CHILDREN,MUTATE_PROB)
pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in current_population]

item#    weight  value
1 	 0.10 	 4.99
2 	 0.33 	 19.02
3 	 0.57 	 7.54
4 	 0.62 	 9.40
5 	 0.76 	 5.39
6 	 0.68 	 15.42
7 	 0.67 	 16.09
8 	 0.16 	 6.94
9 	 0.63 	 13.71
10 	 0.07 	 19.13
starting maximum: 55.87601735325436


In [ ]:
max(pop_fitness)

63.78539053437586

Here we have an example output of a GA. We created the initial population, examined the maximum value of the ramdomly generated solutions. After running the GA for the first 100,000 iterations, we should have noticed an improvemen in our answer. Try to create your own solution and see if you can beat the current best solution.

Let's run the GA for a couple more iterations and see if things get any better.

In [ ]:
for _ in range(NUM_ITER):
  current_population = ga_iteration(current_population,NUM_CHILDREN,MUTATE_PROB)
pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]
max(pop_fitness)

63.78539053437586

In [ ]:
current_population

[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0,

Our answers should converge. If you suspect that you might be stuck on a local maximum instead of the global maximum, don't be afraid to modify the algorithm. You can change the mutation probability, crossover rules or selection algorithms to see if you get a different result. If all of your modifications yield the same result, you can be pretty convinced that you have found the optimal solution.

here's the full code below. Switches and alternate methods are omitted for simplicity.


In [14]:
import random

# Create knapsack parameters
NUM_ITEMS = 10
MAX_WEIGHT = 2
values = [random.uniform(1,20) for _ in range(NUM_ITEMS)]
weights = [random.random() for _ in range(NUM_ITEMS)]
print(f"item#    weight  value")
for i,w,v in zip(range(NUM_ITEMS),weights,values):
  print(f"{i+1} \t {w:0.2f} \t {v:0.2f}")
#set up the GA
POPULATION_SIZE = 50
#initial population

def knapsack_fitness(solution, values, weights, max_weight):
  total_weight = 0
  total_value = 0
  for choice,weight,value in zip(solution,weights,values):
    total_weight += choice*weight
    total_value += choice*value
  return total_value if total_weight <= max_weight else max_weight-total_weight

def random_knapsack_solution(num_items):
  return [random.randint(0,1) for _ in range(num_items)]

def generate_knapsack_population(num_items, population_size):
    population = [random_knapsack_solution(num_items) for _ in range(population_size)]
    return population

initial_population = generate_knapsack_population(NUM_ITEMS,POPULATION_SIZE)

print(f"starting maximum: {max([knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in initial_population])}") 

initial_population = generate_knapsack_population(NUM_ITEMS,POPULATION_SIZE)
#rules for reproduction

def parent_selection(population):
  return random.sample(population,2)

def one_point_crossover(parent1,parent2):
  i = random.randrange(len(parent1))
  child1 = parent1[:i]+parent2[i:]
  child2 = parent1[i:]+parent2[:i]
  return [child1,child2]

def mutate_knapsack(solution,prob):
  return [s^1 if random.random() <prob else s for s in solution]

def generate_children(population,num_children,mutate_prob):
  children = []
  for _ in range(num_children//2):
    parent1,parent2 = parent_selection(population)
    children += one_point_crossover(parent1,parent2)
  return [mutate_knapsack(child,mutate_prob) for child in children]

NUM_CHILDREN = 50
MUTATE_PROB = 0.1

current_population = initial_population

def tournament_selection(new_population,population_size,pop_fitness):
  trimmed_population = []
  for _ in range(population_size):
    (strat1,fit1),(strat2,fit2) = random.sample(list(zip(new_population,pop_fitness)),2)
    trimmed_population.append(strat1) if fit1>fit2 else trimmed_population.append(strat2)
  return trimmed_population

def ga_iteration(population, num_children,mutate_prob):
  children = generate_children(population,NUM_CHILDREN,MUTATE_PROB)
  new_population = population + children
  pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in population]
  return tournament_selection(new_population,POPULATION_SIZE,pop_fitness)


NUM_ITER = 10000
for _ in range(NUM_ITER):
  current_population = ga_iteration(current_population,NUM_CHILDREN,MUTATE_PROB)
pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in current_population]
print(f"new max fitness: {max(pop_fitness)}")
NUM_ITER = 100
for _ in range(NUM_ITER):
  current_population = ga_iteration(current_population,NUM_CHILDREN,MUTATE_PROB)
pop_fitness = [knapsack_fitness(solution, values, weights, MAX_WEIGHT) for solution in current_population]
print(f"new max fitness: {max(pop_fitness)}")

item#    weight  value
1 	 0.53 	 12.26
2 	 0.14 	 14.43
3 	 0.82 	 1.61
4 	 0.21 	 11.22
5 	 0.44 	 2.56
6 	 0.30 	 18.42
7 	 0.69 	 9.18
8 	 0.47 	 5.51
9 	 0.56 	 17.34
10 	 0.25 	 8.61
starting maximum: 82.27238207073272
new max fitness: 67.96901921914996
new max fitness: 67.96901921914996
